# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: No module named 'pymorphy2'

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

In [3]:
from random import randint
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
import scipy
import pandas as pd
import itertools
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DateBack\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DateBack\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DateBack\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [4]:
preDescriptions_df = pd.read_csv('data/preprocessed_descriptions.csv')
preDescriptions_df.head()

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...


In [5]:
preDescriptions = preDescriptions_df['preprocessed_descriptions'].astype(str).tolist()
notUniqueWords = list(itertools.chain(*[nltk.word_tokenize(description) for description in preDescriptions]))
words = list(set(itertools.chain(*[nltk.word_tokenize(description) for description in preDescriptions])))
len(words)

45416

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [6]:
preDescriptionsWordsPairs = [[words[randint(0, len(words))] for j in range(2)] for i in range(5)]
for pair in preDescriptionsWordsPairs:
    print(pair[0], pair[1], nltk.edit_distance(pair[0], pair[1]))

samples shellthe 6
toyed gruyere 5
creminiand fernandina 8
zaatar mostest 6
deliciousof 2lb 10


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [7]:
d = {}
def find_nearest_words(targetWord, k):
    for word in words:
        if word != targetWord:
            d[word] = nltk.edit_distance(word, targetWord)
        
    for w in sorted(d, key=d.get)[:k]:
        print(w, d[w])
    
find_nearest_words('recommend', 4)

recommends 1
recommendi 1
recommended 2
recommendsi 2


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [8]:
stemmer = SnowballStemmer('english')
wnl = nltk.WordNetLemmatizer()

stemmLemantiz_df = pd.DataFrame(data={
    'word': words,
    'stemmed_word': [stemmer.stem(word) for word in words],
    'normalized_word': [wnl.lemmatize(word) for word in words]
}).set_index('word')


stemmLemantiz_df_notUnique = pd.DataFrame(data={
    'word': notUniqueWords,
    'stemmed_word': [stemmer.stem(word) for word in notUniqueWords],
    'normalized_word': [wnl.lemmatize(word) for word in notUniqueWords]
}).set_index('word')


stemmLemantiz_df.head()

,stemmed_word,normalized_word
word,,
tread,tread,tread
slowfoodbeirutorgindexinvphpcinv18,slowfoodbeirutorgindexinvphpcinv18,slowfoodbeirutorgindexinvphpcinv18
concauction,concauct,concauction
perfectit,perfectit,perfectit
cookiesenjoy,cookiesenjoy,cookiesenjoy


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [9]:
StopWords_df = stemmLemantiz_df_notUnique[stemmLemantiz_df_notUnique.index.isin(stopwords.words('english'))]
NormalWords_df = stemmLemantiz_df_notUnique[~stemmLemantiz_df_notUnique.index.isin(stopwords.words('english'))]
print('Доля стоп слов', StopWords_df.shape[0] / NormalWords_df.shape[0])

Доля стоп слов 0.8198634124030686


In [10]:
print('Часто употребляемы до удаления')
stemmLemantiz_df_notUnique.index.value_counts()[:10]

Часто употребляемы до удаления


the     38827
a       34630
and     30058
this    25278
to      23371
i       21797
is      20222
of      18307
it      18036
for     15737
Name: word, dtype: int64

In [11]:
print('Часто употребляемы после удаления')
NormalWords_df.index.value_counts()[:10]

Часто употребляемы после удаления


recipe    14239
make       6135
time       4931
use        4484
great      4225
like       4100
easy       4024
made       3765
one        3749
good       3542
Name: word, dtype: int64

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [12]:
preDescriptions_sample = preDescriptions_df.sample(5)
vectorizer = TfidfVectorizer()
print(vectorizer.fit_transform(preDescriptions_sample['preprocessed_descriptions']).toarray())

[[0.28150333 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.28150333 0.         0.
  0.         0.         0.         0.         0.         0.28150333
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.15859407 0.         0.28150333 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.28150333 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.      

3.2 Вычислите близость между каждой парой рецептов, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [18]:
# Сделано на основе пяти случайных рецептов а не всего датасета для ускорения
data = squareform(pdist(vectorizer.fit_transform(preDescriptions_sample['preprocessed_descriptions']).toarray(), metric='cosine'))
buffer_df = pd.DataFrame(data,
                         columns=preDescriptions_sample['name'].values,
                         index=preDescriptions_sample['name']
                        )
buffer_df

,crushed potatoes,wing bean and grilled prawn salad,salmon with couscous vegetable salad,peanut butter and milk chocolate chip tassies,delicious coconut custard pie
name,,,,,
crushed potatoes,0.000000,0.989759,0.982755,0.952457,0.957467
wing bean and grilled prawn salad,0.989759,0.000000,0.717762,0.891649,0.907790
salmon with couscous vegetable salad,0.982755,0.717762,0.000000,0.856142,0.884180
peanut butter and milk chocolate chip tassies,0.952457,0.891649,0.856142,0.000000,0.910055
delicious coconut custard pie,0.957467,0.907790,0.884180,0.910055,0.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат

In [20]:
# Рецепты "salmon with couscous vegetable salad" и "wing bean and grilled prawn salad" максимально схожи
# из за наименьшего косинусного расстояния 0.717762
# При перезапуске могут быть другие рецепты из-за рандомной выборки рецептов